In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
#import nltk
#nltk.download('wordnet')
import regex
import spacy
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
df = pd.read_csv('tweets.csv')

In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [5]:
neg = df[df['airline_sentiment'] == 'negative']
pos = df[df['airline_sentiment'] == 'positive']
pos_len = len(pos)
neg_len = len(neg)
sentisize = len(df['airline_sentiment'])
print("Total number of negative sentiments: " + str(neg_len))
print("Total number of positive sentiments: " + str(pos_len))
print("The total entries in the field 'airline sentiment':" + str(sentisize))

Total number of negative sentiments: 9178
Total number of positive sentiments: 2363
The total entries in the field 'airline sentiment':14640


The central limit theorem states that the sampling distribution of the mean of any independent, random variable will be normal or nearly normal, if the sample size is large enough.

Central Limit Theorem assumes the following conditions:

i) Sample size should be "large enough".

Since the data is large ( n > 30), this condition is met.

ii) np>= 10 and nq >= 10. The number of successes and failures of the samples should be greater than or equal to 10.

Success: There are 2363 positive sentiments out of 14640 > 10. Similarly, there are 9178 negative sentiments out of 14640 > 10.

Failure: There are 12277(14640 - 2363) sentiments which are not positive > 10. Similarly, there are 5462(14640 - 9178) which are negative > 10.

Therefore, this condition is met.

iii) Independence. Sample size should be less than or equal to 10% of the population size. We have seen this already in (i), hence this condition is also met.

iv) Randomization. Assuming that the samples are randomly chosen unless and until it is specified.

Hence CLT is applicable in this scenario and it is appropriate to use a significance test.

Null Hypothesis: Proportion of positive sentiments is equal to proportion of negative sentiments.

Alternate Hypothesis: Proportion of positive sentiments is not equal to proportion of negative sentiments.

Let us try Bootstraping two sample test:


In [6]:
alpha = 0.05
empirical_diff = np.mean(neg.airline_sentiment_confidence) - np.mean(pos.airline_sentiment_confidence)

def bootstrap_sample(data,func):
    return func(np.random.choice(data,size=len(data)))


def bootstrap_replicate(data,func,size=1):
    
    bs_replicate = np.empty(size)
    
    for i in range(size):
        bs_replicate[i] = bootstrap_sample(data,func)
        
    return bs_replicate


shifted_pos = pos.airline_sentiment_confidence - np.mean(pos.airline_sentiment_confidence) + np.mean(df.airline_sentiment_confidence)
shifted_neg = neg.airline_sentiment_confidence - np.mean(neg.airline_sentiment_confidence) + np.mean(df.airline_sentiment_confidence)


pos_bs_replicate = bootstrap_replicate(shifted_pos,np.mean,10000)
neg_bs_replicate = bootstrap_replicate(shifted_neg,np.mean,10000)

bs_replicate_diff = pos_bs_replicate - neg_bs_replicate

p = np.sum(bs_replicate_diff >= empirical_diff)/len(bs_replicate_diff)

if p <= alpha:
    print("We can reject the null hypothesis with p-value:" + str(p))
else:
    print("We cannot reject the null hypothesis with p-value: " + str(p))

We can reject the null hypothesis with p-value:0.0


Let us now try the frequentist approach:

In [7]:
pos_prop = (pos_len/sentisize)
neg_prop = (neg_len/sentisize)
print("Proportion of positive sentiments (P1): %.2f" % pos_prop)
print("Proportion of negative sentiments (P2): %.2f" % neg_prop)

Proportion of positive sentiments (P1): 0.16
Proportion of negative sentiments (P2): 0.63


Since the inference conditions are met and sample size is large enough (n > 30), two sample Z-test will be performed.

z statistic for two sample is calculated as P1 - P2/sigma1 - sigma2

sigma = sqrt(Pc * (1-Pc) * (1/n1 + 1/n2)) where Pc is combined proportion; n1 and n2 are totals of each proportion.

In [8]:
Pc = (pos_len + neg_len)/(sentisize + sentisize)
print("Combined sample proportion is " + str(Pc))
StandardError = np.sqrt(Pc * (1- Pc) * (1/sentisize + 1/sentisize))
print("Standard Error : " + str(StandardError))

Combined sample proportion is 0.39415983606557375
Standard Error : 0.005711624850139768


In [9]:
from scipy import stats

z_statistic = (neg_prop - pos_prop)/StandardError


p = stats.norm.sf(abs(z_statistic))

print("p-value is : %.5f" % p )

if p <= alpha:
    print("We can reject the null hypothesis.")
else:
    print("We cannot reject the null hypothesis.")

p-value is : 0.00000
We can reject the null hypothesis.


After simulating the conditions for 10,000 trials, we got the same results following both the bootstrap and frequentist approaches, hereby concluding that the proportion of negative sentiments is NOT equal to positive sentiments.